In [ ]:
# api_emporio.py
# API REST para servir el modelo de churn de Emporio Vinos y Licores

from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
from joblib import load
import numpy as np
import csv
from datetime import datetime


# Cargar artefacto entrenado
artifact = load("/content/drive/MyDrive/Colab Notebooks/EmporioIA/EmporioIA/modelo_churn_rf_v1.joblib")
model = artifact["model"]
feature_cols = artifact["feature_cols"]
threshold_top10 = artifact["threshold_top10"]

# IMPORTANTE:
# feature_cols fue generado por get_dummies(region, canal, drop_first=True)
# y el resto de features numéricos. Vamos a mapear manualmente los campos
# de entrada a esas columnas.

# Lista estándar de columnas esperadas (en orden lógico)
# Ajusta si en tu script original agregaste más columnas.
FEATURE_ORDER = [
    "freq_180",
    "spend_total",
    "spend_avg",
    "coupon_rate",
    "recency_days",
    "tenure_days",
    "arpu",
    "share_Accesorio",
    "share_Cerveza",
    "share_Destilado",
    "share_Vino",
    "region_Norte",
    "region_Sur",
    "canal_Web",
    "canal_WhatsApp",
]

# Esquema de entrada de la API (lo que recibe el JSON)
class ClienteFeatures(BaseModel):
    freq_180: int
    spend_total: float
    spend_avg: float
    coupon_rate: float
    recency_days: int
    tenure_days: int
    arpu: float
    share_Accesorio: float
    share_Cerveza: float
    share_Destilado: float
    share_Vino: float
    region: str         # "Norte", "Centro" o "Sur"
    canal: str          # "Tienda", "Web" o "WhatsApp"

app = FastAPI(
    title="API Churn Emporio Vinos y Licores",
    version="1.0.0",
    description="Servicio de predicción de no recompra (churn 30 días).",
)

def encode_region(region: str):
    region = region.capitalize()
    region_norte = 1 if region == "Norte" else 0
    region_sur   = 1 if region == "Sur" else 0  # Centro => (0,0)
    return region_norte, region_sur

def encode_canal(canal: str):
    # Base es Tienda (0,0)
    canal = canal.capitalize()
    canal_web = 1 if canal == "Web" else 0
    canal_wa  = 1 if canal in ["Whatsapp", "WhatsApp"] else 0
    return canal_web, canal_wa

LOG_FILE = "logs_churn.csv"

def log_prediction(row_dict, proba_churn, riesgo):
    """
    Guarda un registro mínimo de cada predicción en un CSV para monitoreo.
    """
    campos = [
        "timestamp",
        "proba_churn",
        "riesgo",
        "freq_180",
        "spend_total",
        "recency_days",
        "tenure_days",
        "region",
        "canal",
    ]

    # Si el archivo está vacío, escribimos encabezado
    try:
        nuevo_archivo = not os.path.exists(LOG_FILE)
    except NameError:
        import os
        nuevo_archivo = not os.path.exists(LOG_FILE)

    with open(LOG_FILE, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        if nuevo_archivo:
            writer.writerow(campos)

        writer.writerow([
            datetime.now().isoformat(timespec="seconds"),
            proba_churn,
            riesgo,
            row_dict["freq_180"],
            row_dict["spend_total"],
            row_dict["recency_days"],
            row_dict["tenure_days"],
            row_dict["region"],
            row_dict["canal"],
        ])


@app.post("/predict")
def predict_churn(cliente: ClienteFeatures):
    try:
        # One-hot de region y canal
        region_norte, region_sur = encode_region(cliente.region)
        canal_web, canal_wa = encode_canal(cliente.canal)

        # Construir vector en el mismo orden que FEATURE_ORDER
        row = {
            "freq_180": cliente.freq_180,
            "spend_total": cliente.spend_total,
            "spend_avg": cliente.spend_avg,
            "coupon_rate": cliente.coupon_rate,
            "recency_days": cliente.recency_days,
            "tenure_days": cliente.tenure_days,
            "arpu": cliente.arpu,
            "share_Accesorio": cliente.share_Accesorio,
            "share_Cerveza": cliente.share_Cerveza,
            "share_Destilado": cliente.share_Destilado,
            "share_Vino": cliente.share_Vino,
            "region_Norte": region_norte,
            "region_Sur": region_sur,
            "canal_Web": canal_web,
            "canal_WhatsApp": canal_wa,
            #para log
            "region": cliente.region,
            "canal": cliente.canal,
        }

        # Pasar a vector numpy ordenado
        x_vec = np.array([[row[col] for col in FEATURE_ORDER]], dtype=float)

        # Predicción de probabilidad de churn (clase 1)
        proba_churn = float(model.predict_proba(x_vec)[0, 1])

        # Clasificación de riesgo usando el umbral Top 10%
        riesgo = "ALTO" if proba_churn >= threshold_top10 else "BAJO"

        #log de monitoreo basico
        log_prediction(row, proba_churn, riesgo)

        return {
            "proba_churn": proba_churn,
            "riesgo": riesgo,
            "threshold_top10": float(threshold_top10),
        }

    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))
    






# PRUEBAS UNITARIAS BÁSICAS (encode y salida del modelo)

from math import isclose

# 1) Probar encode_region
assert encode_region("Norte") == (1, 0)
assert encode_region("Sur")   == (0, 1)
assert encode_region("Centro") == (0, 0)
assert encode_region("  sur  ") == (0, 1)

print("✅ encode_region pasa pruebas básicas")

# 2) Probar encode_canal
assert encode_canal("Web") == (1, 0)
assert encode_canal("WhatsApp") == (0, 1)
assert encode_canal("Tienda") == (0, 0)
assert encode_canal("whatsapp") == (0, 1)

print("✅ encode_canal pasa pruebas básicas")

# 3) Probar que el modelo devuelve una probabilidad válida (entre 0 y 1)
ejemplo_row = {
    "freq_180": 5,
    "spend_total": 120000,
    "spend_avg": 24000,
    "coupon_rate": 0.2,
    "recency_days": 15,
    "tenure_days": 240,
    "arpu": 15000,
    "share_Accesorio": 0.05,
    "share_Cerveza": 0.25,
    "share_Destilado": 0.20,
    "share_Vino": 0.50,
    "region_Norte": 0,
    "region_Sur": 0,
    "canal_Web": 1,
    "canal_WhatsApp": 0,
}

import numpy as np

x_vec = np.array([[ejemplo_row[col] for col in FEATURE_ORDER]], dtype=float)
proba = float(model.predict_proba(x_vec)[0, 1])

assert 0.0 <= proba <= 1.0

print(f"✅ predict_proba devuelve valor válido: {proba:.4f}")
print("✅ Pruebas unitarias básicas OK")



# PRUEBA DE INTEGRACIÓN: llamar al endpoint /predict desde código

from fastapi.testclient import TestClient

client = TestClient(app)

payload = {
    "freq_180": 5,
    "spend_total": 120000,
    "spend_avg": 24000,
    "coupon_rate": 0.2,
    "recency_days": 15,
    "tenure_days": 240,
    "arpu": 15000,
    "share_Accesorio": 0.05,
    "share_Cerveza": 0.25,
    "share_Destilado": 0.20,
    "share_Vino": 0.50,
    "region": "Centro",
    "canal": "Web"
}

resp = client.post("/predict", json=payload)

print("Status code:", resp.status_code)
print("Respuesta JSON:", resp.json())

assert resp.status_code == 200
assert "proba_churn" in resp.json()
assert "riesgo" in resp.json()

print("✅ Prueba de integración OK: flujo completo /predict")



# PRUEBAS UNITARIAS BÁSICAS

# 1) encode_region
assert encode_region("Norte") == (1, 0)
assert encode_region("Sur")   == (0, 1)
assert encode_region("Centro") == (0, 0)
assert encode_region("  sur  ") == (0, 1)
print("✅ encode_region OK")

# 2) encode_canal
assert encode_canal("Web") == (1, 0)
assert encode_canal("WhatsApp") == (0, 1)
assert encode_canal("Tienda") == (0, 0)
assert encode_canal("whatsapp") == (0, 1)
print("✅ encode_canal OK")

# 3) modelo devuelve una probabilidad válida
ejemplo_row = {
    "freq_180": 5,
    "spend_total": 120000,
    "spend_avg": 24000,
    "coupon_rate": 0.2,
    "recency_days": 15,
    "tenure_days": 240,
    "arpu": 15000,
    "share_Accesorio": 0.05,
    "share_Cerveza": 0.25,
    "share_Destilado": 0.20,
    "share_Vino": 0.50,
    "region_Norte": 0,
    "region_Sur": 0,
    "canal_Web": 1,
    "canal_WhatsApp": 0,
}

x_vec = np.array([[ejemplo_row[col] for col in FEATURE_ORDER]], dtype=float)
proba = float(model.predict_proba(x_vec)[0, 1])
assert 0.0 <= proba <= 1.0
print(f"✅ predict_proba devuelve valor válido: {proba:.4f}")

print("✅ Pruebas unitarias básicas OK")
